# Esure Assignment -- Geoff Chambers -- 03/02/2021
## 2. Statistical Relationships Between Features and Target Variable

**Objective:** First-pass indication of relative importance of features for predicting target 
- Statistical tests for strength of relationships between target variable and other features
- Visualise relationships between variables

---

In [1]:
### Environment: conda proj-home-ins

# Extensions & config
%load_ext autoreload
%autoreload 1

# General package imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from scipy.stats import ttest_ind, chi2_contingency

## LOAD CLEAN DATA FROM PREVIOUS NOTEBOOK

In [2]:
# Restore stored dataframe

%store -r policies_df

display(policies_df.head(3))
display(policies_df.info())

,claim3years,p1_emp_status,p1_pt_emp_status,bus_use,ad_buildings,risk_rated_area_b,sum_insured_buildings,ncd_granted_years_b,ad_contents,risk_rated_area_c,...,mta_fap,mta_aprp,last_ann_prem_gross,cover_start_y,cover_start_m,cover_start_d,quote_cover_days,cover_mta_days,p1_age_yrs,lapsed
police,,,,,,,,,,,,,,,,,,,,,
P000001,False,R,NaN,False,True,19.0,1000000.0,7.0,True,6.0,...,NaN,NaN,274.81,2007.0,11,22,0.0,NaN,68.438056,True
P000002,False,E,NaN,True,True,25.0,1000000.0,6.0,True,9.0,...,308.83,-9.27,308.83,2008.0,1,1,40.0,NaN,37.618070,False
P000003,False,E,NaN,False,False,NaN,0.0,0.0,True,12.0,...,52.65,52.65,52.65,2007.0,11,23,0.0,1076.0,60.454483,False


<class 'pandas.core.frame.DataFrame'>
Index: 189021 entries, P000001 to P256136
Data columns (total 64 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   claim3years             189021 non-null  bool    
 1   p1_emp_status           189021 non-null  object  
 2   p1_pt_emp_status        1782 non-null    object  
 3   bus_use                 189021 non-null  bool    
 4   ad_buildings            189021 non-null  bool    
 5   risk_rated_area_b       140876 non-null  float64 
 6   sum_insured_buildings   189021 non-null  float64 
 7   ncd_granted_years_b     189021 non-null  float64 
 8   ad_contents             189021 non-null  bool    
 9   risk_rated_area_c       180290 non-null  float64 
 10  sum_insured_contents    189021 non-null  float64 
 11  ncd_granted_years_c     189021 non-null  float64 
 12  contents_cover          189021 non-null  bool    
 13  buildings_cover         189021 non-null  bool    
 14  sp

None

In [3]:
target_var = 'lapsed'

In [4]:
target_base_rate = policies_df[target_var].mean() # overall proportion of lapsed=True

In [5]:
num_features = policies_df.select_dtypes('number').columns
cat_features = policies_df.select_dtypes(exclude='number').drop(columns=[target_var]).columns

## STATISTICAL RELATIONSHIP BETWEEN TARGET AND NUMERIC FEATURES
**2-sample t-test**
- H0: lapsed/unlapsed policies have the same expected value of numeric feature
- Smaller p-values may indicate a stronger relationship between numeric feature and lapsed status (target); large p-values may indicate numeric features with little effect on lapsed status
- This should be taken as a first-pass guide only, because of the possibility of collinearity/confounders, and potential violated assumptions of test (i.e., approx normal sampling distribution)

In [6]:
def get_outlier_mask(ser, n_iqr=1.5):
    """Returns a boolean series of outliers (True=outlier) for input series
    Based on IQR test, but will not remove values inside p1-p99 range,
        nor will it modify Series if >=98% of values are identical
    """
    p1, q1, q3, p99 = ser.quantile([0.01, 0.25, 0.75, 0.99])
    iqr = q3 - q1
    thres_lo = min(q1 - 1.5*iqr, p1)
    thres_hi = max(q3 + 1.5*iqr, p99)
    if thres_lo == thres_hi: # Series is essentially a single value, leave it alone
        return pd.Series(False, index=ser.index)
    outlier_mask = ~ser.between(thres_lo, thres_hi)
    return outlier_mask

In [7]:
# Get test p-value for each numeric feature

num_p_values = {}
for feature in num_features:
    outlier_mask = get_outlier_mask(policies_df[feature])
    
    unlapsed_vals = policies_df.loc[~policies_df[target_var] & ~outlier_mask, feature].dropna().values
    lapsed_vals = policies_df.loc[policies_df[target_var] & ~outlier_mask, feature].dropna().values
    
    stat, pval = ttest_ind(unlapsed_vals, lapsed_vals)
    num_p_values[feature] = pval
    
num_p_values = pd.Series(num_p_values).sort_values()

print("Rough ranking of influence of numeric variables on lapsed status, with p-value of test:")
display(num_p_values)

Rough ranking of influence of numeric variables on lapsed status, with p-value of test:


cover_start_y             0.000000e+00
sum_insured_buildings     0.000000e+00
ncd_granted_years_b       0.000000e+00
max_days_unocc            0.000000e+00
bedrooms                 3.236391e-253
risk_rated_area_b        1.482539e-175
last_ann_prem_gross      4.835772e-164
risk_rated_area_c        5.631947e-102
spec_sum_insured          4.794185e-70
spec_item_prem            1.278767e-64
mta_fap                   7.503673e-53
cover_mta_days            5.342020e-36
yearbuilt                 9.461979e-16
sum_insured_contents      4.935097e-09
quote_cover_days          4.300912e-08
mta_aprp                  5.217341e-07
unspec_hrp_prem           2.341078e-03
ncd_granted_years_c       9.245278e-03
p1_age_yrs                2.624979e-02
paying_guests             5.290710e-02
listed                    1.155666e-01
payment_frequency                  NaN
dtype: float64

## STATISTICAL RELATIONSHIP BETWEEN TARGET AND CATEGORICAL FEATURES
**chi-squared test**
- H0: policy status (lapsed/unlapsed) is independent of categorical feature
- Smaller p-values suggest a stronger relationship between numeric feature and lapsed status (target); large p-values may indicate numeric features with little effect on lapsed status
- This should be taken as a first-pass guide only, because of the possibility of confounders, and potential violated assumptions of test (i.e., insufficient expected frequency for rare categories)

In [8]:
cat_features

Index(['claim3years', 'p1_emp_status', 'p1_pt_emp_status', 'bus_use',
       'ad_buildings', 'ad_contents', 'contents_cover', 'buildings_cover',
       'p1_mar_status', 'p1_policy_refused', 'p1_sex', 'appr_alarm',
       'appr_locks', 'roof_construction', 'wall_construction', 'flooding',
       'neigh_watch', 'occ_status', 'ownership_type', 'prop_type',
       'safe_installed', 'sec_disc_req', 'subsidence', 'payment_method',
       'legal_addon_pre_ren', 'legal_addon_post_ren', 'home_em_addon_pre_ren',
       'home_em_addon_post_ren', 'garden_addon_pre_ren',
       'garden_addon_post_ren', 'keycare_addon_pre_ren',
       'keycare_addon_post_ren', 'hp1_addon_pre_ren', 'hp1_addon_post_ren',
       'hp2_addon_pre_ren', 'hp2_addon_post_ren', 'hp3_addon_pre_ren',
       'hp3_addon_post_ren', 'mta_flag', 'cover_start_m', 'cover_start_d'],
      dtype='object')

In [9]:
# Get test p-value for each categorical feature

cat_p_values = {}
for feature in cat_features:
    cont_table = pd.crosstab(policies_df[feature], policies_df[target_var])
    
    chi2, pval, dof, expected_table = chi2_contingency(cont_table)
    cat_p_values[feature] = pval
    
cat_p_values = pd.Series(cat_p_values).sort_values()

print("Rough ranking of influence of categorical variables on lapsed status, with p-value of test:")
display(cat_p_values)

Rough ranking of influence of categorical variables on lapsed status, with p-value of test:


p1_mar_status              0.000000e+00
payment_method             0.000000e+00
legal_addon_post_ren       0.000000e+00
ad_buildings               0.000000e+00
contents_cover             0.000000e+00
prop_type                  0.000000e+00
hp2_addon_post_ren        2.881593e-170
p1_sex                    6.493297e-149
garden_addon_post_ren     1.305646e-148
hp1_addon_post_ren         2.028934e-87
ownership_type             4.305523e-74
cover_start_d              6.983088e-63
keycare_addon_post_ren     9.190361e-46
garden_addon_pre_ren       3.857626e-44
cover_start_m              4.195952e-42
hp3_addon_post_ren         3.722727e-39
p1_emp_status              2.952771e-35
bus_use                    1.809021e-34
safe_installed             8.949317e-31
flooding                   8.897101e-27
appr_locks                 1.098769e-26
sec_disc_req               5.587765e-19
legal_addon_pre_ren        1.829357e-18
occ_status                 3.458250e-18
p1_policy_refused          6.185678e-10


## VISUALISE RELATIONSHIPS

In [10]:
def plot_relship_num(feature, outlier_filter=True):
    """Visualise relationship between numeric feature and lapsed status
    Produces KDE and boxplot of numeric feature values, split by lapsed status
    Returns (fig, axes)
    """
    if outlier_filter:
        outlier_mask = get_outlier_mask(policies_df[feature])
        plot_data = policies_df.loc[~outlier_mask, [feature, target_var]]
    else:
        plot_data = policies_df[[feature, target_var]]
        
    fig, axes = plt.subplots(2, 1, figsize=(8, 6), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
    
    # KDE on top axis
    sns.kdeplot(plot_data.loc[~plot_data[target_var], feature], ax=axes[0], label='False', shade=True)
    sns.kdeplot(plot_data.loc[plot_data[target_var], feature], ax=axes[0], label='True', shade=True)
    axes[0].legend(title=target_var)
    
    # Boxplots on bottom axis
    sns.boxplot(x=feature, y=target_var, data=plot_data, ax=axes[1], orient='h', boxprops={'alpha': 0.6})
    
    for ax in axes:
        ax.grid(axis='x')
    
    return (fig, axes)

In [11]:
def plot_relship_cat(feature):
    """Plot relationship between categorical feature and lapsed status
    Produces stacked bar plots of contingency table, unnormalised (record count) and normalised (relative frequency)
    Returns (fix, axes)
    """
    cont_table = pd.crosstab(policies_df[feature], policies_df[target_var])
    cont_table_rel = cont_table.div(cont_table.sum(axis=1), axis=0)
    
    fig, axes = plt.subplots(2, 1, figsize=(8, 6), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
    
    # contingency table record counts on top axis
    cont_table.plot(ax=axes[0], kind='bar', stacked=True, alpha=0.6)
    axes[0].set_ylabel('Record count')
    axes[0].legend(title=target_var)
    
    # lapsed ratio for each feature category
    cont_table_rel.plot(ax=axes[1], kind='bar', stacked=True, alpha=0.6, legend=None)
    axes[1].set_ylabel('Relative frequency')
    axes[1].axhline(y=1-target_base_rate, color='red', linestyle='--')
    
    for ax in axes:
        ax.grid('y')
    
    return (fig, axes)

In [12]:
# Interactive plotter

p_values = num_p_values.append(cat_p_values).sort_values() # combined p-values

def interactive_relship_plot(feature, show_fig=True):
    """Interactive plotter for relationship of feature to target variable
    """
    if feature in num_features:
        fig, axes = plot_relship_num(feature)
    elif feature in cat_features:
        fig, axes = plot_relship_cat(feature)
    else:
        raise ValueError(f"Provided feature={feature} not found in either numeric or categorical feature lists")
        
    fig.suptitle(f"Distribution of {feature} relative to {target_var} (p-value {p_values[feature]:.1e})")
    fig.tight_layout()
    if show_fig:
        plt.show()
    else:
        return fig, axes

feature_sel = widgets.Dropdown(options=p_values.index, description='Feature:', style={'description_width': 'initial'})
widgets.interact(interactive_relship_plot, feature=feature_sel, show_fig=widgets.fixed(True))

interactive(children=(Dropdown(description='Feature:', options=('cover_start_y', 'legal_addon_post_ren', 'ad_b…

<function __main__.interactive_relship_plot(feature, show_fig=True)>

### *Observations:*
- More recent policies (cover_start_y) are more likely to lapse
- Non-direct-debit are more likely to lapse than direct-debit (payment_method)
- Marital status (p1_mar_status) seems to have an effect, although cannot say much about it without knowing the codes
- Last annual premium (last_ann_premium_gross) has perhaps less overall effect than expected, although low (<100) premiums are less likely to lapse
- Slight effect of number of bedrooms (bedrooms); larger houses slightly more likely to lapse
- Prior claims history (claim3years) has little effect

In [13]:
# Save specified images to file

save_features = [
    'cover_start_y',
    'payment_method',
    'p1_mar_status',
    'bedrooms',
    'last_ann_prem_gross',
    'claim3years'
]
save_features = None

save_dir = "../images/"

if save_features is not None:
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    
    print("Creating image files:")
    for feature in save_features:
        fname = save_dir + f"{target_var.replace('_', '-')}_vs_{feature.replace('_', '-')}.png"
        fig, axes = interactive_relship_plot(feature, show_fig=False)
        fig.savefig(fname)
        plt.close(fig)
        print(f"  {fname}")

## OLD / BACKUP CELLS

In [14]:
raise RuntimeError('Stop here, dont run old cells')

RuntimeError: Stop here, dont run old cells

In [ ]:
# Plot contingency table delta observed - expected, as fraction of records
#- probably not bother with this

cont_table_exp = pd.DataFrame(exp_table, index=cont_table.index, columns=cont_table.columns)
cont_deltafraction_obs_exp = (cont_table - cont_table_exp) / cont_table.sum().sum()

sns.heatmap(cont_deltafraction_obs_exp.iloc[::-1], annot=True, vmin=-0.5, vmax=0.5, cmap='seismic_r')
plt.show()